Import Libraries

In [ ]:
# Install statsmodels
%pip install statsmodels
%pip install lightgbm
%pip install scikit-learn
%pip install seaborn
%pip install matplotlib
%pip install pandas
%pip install numpy
#Installation of required libraries
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.simplefilter(action = "ignore") 

Import Dataset

In [9]:
#Reading the dataset
df = pd.read_csv("data/diabetes.csv")
# The first 5 observation units of the data set were accessed.
df.head()
# The size of the data set was examined. It consists of 768 observation units and 9 variables.
df.shape
#Feature information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Data Preprocessing

In [18]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE

# Replace zeros in specific columns with median values
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for column in columns_with_zeros:
    df[column] = df[column].replace(0, df[column].median())

# Separate features and target variable
X = df.drop(["Outcome"], axis=1)
y = df["Outcome"]

# Apply MinMaxScaler for normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Convert back to DataFrame with correct column names
X_train_balanced = pd.DataFrame(X_train_balanced, columns=X_train.columns)

# Feature Selection using Recursive Feature Elimination (RFE)
log_reg = LogisticRegression(max_iter=1000, random_state=42)
rfe = RFE(log_reg, n_features_to_select=5)
rfe.fit(X_train_balanced, y_train_balanced)

selected_features = X_train.columns[rfe.support_]
print("Selected Features after RFE:", selected_features)

# Select only the top features from X_train_balanced and X_test
X_train_balanced = X_train_balanced[selected_features]
X_test = X_test[selected_features]

Selected Features after RFE: Index(['Pregnancies', 'Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age'], dtype='object')


In [19]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,30.5,33.6,0.627,50
1,1,85,66,29,30.5,26.6,0.351,31
2,8,183,64,23,30.5,23.3,0.672,32
3,1,89,66,23,94.0,28.1,0.167,21
4,0,137,40,35,168.0,43.1,2.288,33


Base Models

In [20]:
# Splitting data (assuming X and y are already defined)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=12345),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=12345),
    'Random Forest': RandomForestClassifier(random_state=12345),
    'Support Vector Machine': SVC(gamma='auto', random_state=12345),
    'Gradient Boosting (XGB)': GradientBoostingClassifier(random_state=12345)
}

# Train all models
trained_models = {}
for name, model in models.items():
    model.fit(X_train_balanced, y_train_balanced)  # Train the model with balanced data
    trained_models[name] = model  # Store trained model
    print(f"{name} trained successfully!")

Logistic Regression trained successfully!
K-Nearest Neighbors trained successfully!
Decision Tree trained successfully!
Random Forest trained successfully!
Support Vector Machine trained successfully!
Gradient Boosting (XGB) trained successfully!


In [21]:
# Select only the features from the RFE result
selected_features = ['Pregnancies', 'Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# Select a single test entry
single_entry = X_test.iloc[0:1][selected_features]  # Keep DataFrame format and select relevant features
actual_label = y_test.iloc[0]  # Get actual label

# Evaluate single entry with each model
print("\nSingle Entry Predictions:")
for name, model in trained_models.items():
    predicted_label = model.predict(single_entry)
    print(f"{name}: Predicted={predicted_label[0]}, Actual={actual_label}")



Single Entry Predictions:
Logistic Regression: Predicted=1, Actual=0
K-Nearest Neighbors: Predicted=1, Actual=0
Decision Tree: Predicted=1, Actual=0
Random Forest: Predicted=1, Actual=0
Support Vector Machine: Predicted=1, Actual=0
Gradient Boosting (XGB): Predicted=1, Actual=0


In [23]:
# Select multiple test entries
num_samples = 10  # Change this to however many samples you want to test
X_samples = X_test.iloc[:num_samples]  # Select the first `num_samples` entries
y_samples = y_test.iloc[:num_samples]  # Get corresponding actual labels

# Create a DataFrame to store results
results_df = pd.DataFrame(columns=['Sample Index', 'Actual'] + list(trained_models.keys()))

# Evaluate multiple entries with each model
for i, (idx, actual_label) in enumerate(y_samples.items()):
    row = {'Sample Index': idx, 'Actual': actual_label}  # Store sample index and actual label
    for name, model in trained_models.items():
        predicted_label = model.predict([X_samples.iloc[i]])[0]  # Predict for one row
        row[name] = predicted_label  # Store prediction
    
    # Use pd.concat instead of append to add the row to the DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

# Print results
print("\nModel Predictions on Multiple Test Entries:")
print(results_df.to_string(index=False))  # Print without DataFrame index

results_df.to_csv("model_predictions_test.csv", index=False)



Model Predictions on Multiple Test Entries:
Sample Index Actual Logistic Regression K-Nearest Neighbors Decision Tree Random Forest Support Vector Machine Gradient Boosting (XGB)
          44      0                   1                   1             1             1                      1                       1
         672      0                   0                   0             0             0                      0                       0
         700      0                   0                   0             0             0                      0                       0
         630      1                   0                   1             0             0                      0                       0
          81      0                   0                   0             0             0                      0                       0
         389      0                   0                   0             0             1                      0                       0
         3

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

# Split dataset (assumes X and y are defined)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=12345)

# Scale features for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define individual models
models = [
    ('LR', LogisticRegression(random_state=12345)),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier(random_state=12345)),
    ('RF', RandomForestClassifier(n_estimators=200, random_state=12345)),
    ('SVM', SVC(gamma='auto', random_state=12345)),
    ('XGB', GradientBoostingClassifier(n_estimators=200, random_state=12345))
]

# Create a Voting Classifier with majority voting
voting_clf = VotingClassifier(estimators=models, voting='hard')  # 'hard' for majority vote
voting_clf.fit(X_train, y_train)

# Get predictions for multiple test samples
num_samples = 10  # Change this to however many test samples you want
X_samples = X_test[:num_samples]
y_samples = y_test[:num_samples]

# Store results in a DataFrame
results_df = pd.DataFrame(columns=['Sample Index', 'Actual', 'Final Prediction'])

# Evaluate and print results
for i, (idx, actual_label) in enumerate(y_samples.items()):
    final_prediction = voting_clf.predict([X_samples[i]])[0]  # Get majority vote prediction
    row = {'Sample Index': idx, 'Actual': actual_label, 'Final Prediction': final_prediction}
    
    # Use pd.concat to add the row to the DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

# Print results
print("\nMajority Vote Predictions:")
print(results_df.to_string(index=False))  # Print nicely formatted table

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Evaluate performance metrics
y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' averages scores based on class distribution
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

# Print performance metrics
print("\nPerformance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (weighted): {precision:.4f}")
print(f"Recall (weighted): {recall:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")
print("\nConfusion Matrix:")
print(cm)



Majority Vote Predictions:
Sample Index Actual Final Prediction
         245      1                1
         473      0                1
         206      1                1
         204      0                0
         655      1                0
         275      0                0
         583      0                0
         264      1                0
         401      0                0
         122      0                0


NameError: name 'precision_score' is not defined